In [1]:
# import necessary packages
import sys
import os, time
import torch 
import numpy as np
import matplotlib.pyplot as plt 
from torch.utils.data import Dataset, DataLoader, TensorDataset
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms, utils
import random, matplotlib
import pandas as pd
#from torch.utils.tensorboard import SummaryWriter
#from torchvision.models.resnet import BasicBlock
#import torch.utils.model_zoo as model_zoo

import scipy.io
import h5py
import gc
from sklearn.model_selection import train_test_split
from PIL import Image
sys.path.append('./models/')
sys.path.append('./source/')

import utils as myUtils
from DataTypes import EEGDataset
from CNN_1 import Network

# Check does your computer support using GPU
print("GPU_available={}".format(torch.cuda.is_available()))

GPU_available=True


In [2]:


# EEG_file_types = [2,1,0]
EEG_file_types = [2]

for EEG_file_type in EEG_file_types:

    folder = "./Results/trial18/1FC_layer/"

    if(EEG_file_type == 0):
        mat = scipy.io.loadmat('EEG_samples_raw.mat')
        EEG_samples = mat['EEG_samples']
        sub_folder = "Kfold_raw/"
    elif(EEG_file_type == 3):
        sub_folder = "Kfold_noise_pruned/"
    elif(EEG_file_type == 2):
        mat = scipy.io.loadmat('EEG_samples_5_18_pruned.mat')
        EEG_samples = mat['EEG_samples']
        sub_folder = "Kfold_pruned/"
    else:
        mat = scipy.io.loadmat('EEG_samples_roi.mat')
        EEG_samples = mat['EEG_samples']
        sub_folder = "Kfold_roi/"

    folder = folder + sub_folder
    no_participants = 29
    train_data_set, train_label_set = [], []
    epoches = 500

    for partic_id in range(9,no_participants):
        fileName = "Subject_"+str(partic_id+1)+"_Results.txt"
        testFileName = "Test_Results.txt"
        path = folder+fileName
        test_path = folder+testFileName

        # Spilt the data into training and testing sets 
        if(EEG_file_type == 3):
            full_data, train_data, test_data = myUtils.generateTrainTestByFile('EEG_samples_noise_pruned.mat', partic_id, normalize = True)
        else:
            full_data, train_data, test_data = myUtils.generateTrainTest(EEG_samples, partic_id, normalize = True)

        # Save information concerning the data to file
        myUtils.writeStatsToFile(full_data, train_data, test_data, path)

        # Create a torch data loader for both the training and testing sets
        train_loader = myUtils.generateTorchLoaders(train_data[0], train_data[1], EEGDataset)
        test_loader = myUtils.generateTorchLoaders(test_data[0], test_data[1], EEGDataset)

        # Free up memory space
        del full_data
        del train_data
        del test_data
        gc.collect()


        # Create Instance of CNN model
        net = Network(num_classes=2)

        # Train the model
        print("Training on-going for participant "+str(partic_id+1))
        net = myUtils.train(net, train_loader, test_loader, epoches, path)

        #Test the model
        acc_test = myUtils.eval(net, test_loader, test_path)

        # Free up memory space
        del train_loader
        del test_loader
        del net
        gc.collect()



Training on-going for participant 12
Cuda is Avaliable


c:\Users\Abdulsatar\Documents\UBICOMP\Thesis Work\ReverseNeuro\./models\CNN_1.py:60: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax8(x)


epoch: 0, lr: 0.010000, accuracy: 0.498494, loss: 0.693215, valid accuracy: 0.999452
epoch: 1, lr: 0.010000, accuracy: 0.500567, loss: 0.693093, valid accuracy: 0.344298


KeyboardInterrupt: 